In [57]:
import pandas as pd

data = [[1, 'Daniel'], 
        [2, 'Jade'], 
        [3, 'Stella'], 
        [4, 'Jonathan'], 
        [5, 'Will']]
student = pd.DataFrame(data, 
                       columns=['student_id', 
                                'student_name']).astype({'student_id':'Int64', 
                                                         'student_name':'object'})

data = [[10, 1, 70], 
        [10, 2, 80], 
        [10, 3, 90], 
        [20, 1, 80], 
        [30, 1, 70], 
        [30, 3, 80], 
        [30, 4, 90], 
        [40, 1, 60], 
        [40, 2, 70], 
        [40, 4, 80]]
exam = pd.DataFrame(data, 
                    columns=['exam_id', 
                             'student_id', 
                             'score']).astype({'exam_id':'Int64', 
                                               'student_id':'Int64', 
                                               'score':'Int64'})
display(student, exam)

,student_id,student_name
0,1,Daniel
1,2,Jade
2,3,Stella
3,4,Jonathan
4,5,Will


,exam_id,student_id,score
0,10,1,70
1,10,2,80
2,10,3,90
3,20,1,80
4,30,1,70
5,30,3,80
6,30,4,90
7,40,1,60
8,40,2,70
9,40,4,80


In [58]:
# exam["highest"] = exam.groupby("exam_id")["score"].rank(method="dense", ascending=False)
# exam["lowest"] = exam.groupby("exam_id")["score"].rank(method="dense", ascending=True)

# highest_ids = exam[exam["highest"]==1]["student_id"].values.tolist()
# print(highest_ids)
# lowest_ids = exam[exam["lowest"]==1]["student_id"].values.tolist()
# print(lowest_ids)

# remove_ids = list(set(highest_ids+lowest_ids))
# print(remove_ids)

# print(student["student_id"].unique())
# print(exam["student_id"].unique())

# unique_students_s = set(student["student_id"].unique())
# unique_students_e = set(exam["student_id"].unique())
# present_students = unique_students_s.intersection(unique_students_e)

# keep_ids = list(present_students.difference(set(remove_ids)))
# print(keep_ids)

# student = student[student["student_id"].isin(keep_ids)]

# student

In [59]:
exam['highest_score']=exam.groupby('exam_id')['score'].transform('max')
exam['lowest_score']=exam.groupby('exam_id')['score'].transform('min')
exam

,exam_id,student_id,score,highest_score,lowest_score
0,10,1,70,90,70
1,10,2,80,90,70
2,10,3,90,90,70
3,20,1,80,80,80
4,30,1,70,90,70
5,30,3,80,90,70
6,30,4,90,90,70
7,40,1,60,80,60
8,40,2,70,80,60
9,40,4,80,80,60


In [60]:
max_min_students= exam[(exam['score']== exam['highest_score']) | (exam['score']==exam['lowest_score']) ][['student_id']]
max_min_students

,student_id
0,1
2,3
3,1
4,1
6,4
7,1
9,4


In [61]:
exam = exam[~exam['student_id'].isin(max_min_students['student_id'])]
exam

,exam_id,student_id,score,highest_score,lowest_score
1,10,2,80,90,70
8,40,2,70,80,60


In [62]:
merged=exam.merge(student,on='student_id')
merged


,exam_id,student_id,score,highest_score,lowest_score,student_name
0,10,2,80,90,70,Jade
1,40,2,70,80,60,Jade


In [63]:
merged = merged[['student_id','student_name']].drop_duplicates().sort_values(by='student_id')
merged

,student_id,student_name
0,2,Jade
